<font size='6px'> Utils </font> 
<br>
<ol>
    <li> Read Tappy file and user file</li>
    <li> Batch Generator </li>
</ol>

In [2]:
from keras.utils import Sequence
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
import os
import numpy as np
import pandas as pd

In [4]:
def readXY(x_path,y_path):
    
    # read all the files (i.e 8490) in form as array
    X = os.listdir(x_path)
    
    # read excel file as a dataframe
    df_excel = pd.read_excel(y_path)
    
    # convert dataframe to numpy matrix
    Y = df_excel.as_matrix()
    
    return X,Y

In [5]:
def xProcess(_X,gender):
    
    X = []
    
    # datetime.time object
    ptime = _X[0][2]
    for row in _X:
        
        if type(row[2])==type(ptime):
            time1 = row[2].hour*60*60 + row[2].minute*60 + row[2].second + row[2].microsecond/1000000
            time2 = ptime.hour*60*60 + ptime.minute*60 + ptime.second + ptime.microsecond/1000000
            dtime = time1 - time2
            ptime = row[2]
        else:
            print("\n number detected row: \n",row)
        
        #temp= [dtime, gender, row[3], row[4], row[5],row[6:9],row[9:] ]
        #print("row[0] :",type(row[0]))
        #print("row[1] :",type(row[1]))
        #print("row[2] :",type(row[2]))
        #print("row[3] :",type(row[3]))
        #print("row[4] :",type(row[4]))
        #print("row[5] :",type(row[5]))
        #print("row[6] :",type(row[6]))
        #print("row[7] :",type(row[7]))
        #print("row[8] :",type(row[8]))
        #print("row[9] :",type(row[9]))
        #print("row[10] :",type(row[10]))
        #print("row[11] :",type(row[11]))
        #print("row[12] :",type(row[12]))
        #print("row[13] :",type(row[13]))
        
        # Considering dtime that is time interval between register of two keystrokes
        # but values seems redundant because we already have  -> Hold_time,Latency_time,Flight_time
        # temp= np.asarray([dtime,gender])
        
        # gender
        temp= np.asarray([])
        temp = np.concatenate((temp,row[3:]), axis=0)

        X.append(temp)
        
    
    return X
        

def TextProcess(X_arr,Y_dict,x_path):
    
    leng = len(X_arr)
    X = []
    Y = []
    
    for index in range(leng):
        
        # read excel file as a dataframe
        df_excel = pd.read_excel(x_path+X_arr[index])
        
        df_excel = df_excel.dropna(axis=0, how='any')
        
        # normalize selected colum  Hold_time,Latency_time,Flight_time
        cols_to_norm = ['Hold_time','Latency_time','Flight_time']
        #df_excel[cols_to_norm] = df_excel[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
        df_excel[cols_to_norm] = df_excel[cols_to_norm].apply(lambda x: (x - x.mean()) / x.std())
        
        
        #print(df_excel)
        # convert dataframe to numpy matrix
        x_individual = df_excel.as_matrix()
        
        
        # using userid to detect is Y_dict['userId']
        # some case might have userid field error or we might not have userid data
        # hence skip them
        user_id = X_arr[index].split('_')[0]
        #print(user_id)
        if user_id not in Y_dict.keys(): 
            print(x_path+"/"+X_arr[index],x_individual[0][0])
        
        y_app = Y_dict[user_id]

        #print(x_individual[0])

        
        x_individual = xProcess(x_individual,y_app[2])
        
        #print(x_individual)
        
        X.append(np.asarray(x_individual))

        # 3 index -> Parkinsons
        y_value = y_app[3]
        Y.append([ 1 - y_value, y_value])
        
        #break
        
        
    #print(X)
    X = np.array(X)
    Y = np.array(Y,dtype=np.float32)
    return X,Y



In [6]:
# Sequence is from keras
class CustomBatchGenerator(Sequence):
    
    # X : array(["user_YYMMDD.txt",.........])
    # 
    # Y will be array of : array([user,BirthYear,Gender,Parkinsons,Tremors,DiagnosisYear,Levadopa,DA,MAOB,Other,
    #                           Impact_Medium,Impact_Mild,Impact_Severe,Sided_Left,Sided_None,Sided_Right,
    #                           UPDRS_1.0,UPDRS_2.0,UPDRS_3.0,UPDRS_4.0])
    def __init__(self, X,Y, x_path, BATCH_SIZE,display_in=1):
        
        self.BatchS = BATCH_SIZE
        self.display_in = display_in
        self.batch_number = 0
        
        self.X = X
        self.Y = Y
        
        self.x_path = x_path
        
        self.Y_dict = {}
        for row in self.Y:
            self.Y_dict[row[0]]= row
    
    
    # Inhertied from keras.utils.Sequence
    # __len__ function returns number of steps to be taken to complete one epoch
    def __len__(self):
        return int(float(len(self.X)/self.BatchS))

    # Inhertied from keras.utils.Sequence
    # This function extecutes after each epoch
    def on_epoch_end(self):
        print("epoch Ended: ",self.batch_number)
        self.batch_number = 0
        np.random.shuffle(self.X)

    # Inhertied from keras.utils.Sequence
    def __getitem__(self, idx):

        #
        #   __len__ function returns number of steps to be taken to complete one epoch
        #   and idx represents each step
        #   
        #   l_bound and r_bound are to get the certain number of data in form of batches from entire dataset 
        #   given the step count for total number of iterations/ epochs 
        #
        #if self.batch_number%self.display_in==0:
        #    print(" Batch: ", self.batch_number)
        self.batch_number = self.batch_number + 1

        l_bound = idx * self.BatchS
        r_bound = (idx+1) * self.BatchS

        X,Y = TextProcess(self.X[l_bound:r_bound],self.Y_dict,self.x_path)
        
        #print(X.shape)
        
        # output: X,Y
        # All details used for X can be provided users while time of testing for parkisons
        # X will be a array of : array([Gender,Hold_time,Latency_time,Flight_time,
        #                              oneHot[Hand_Kpressed_L,Hand_Kpressed_R,Hand_Kpressed_S],
        #                              oneHot[Direction_LL,Direction_LR,Direction_LS,Direction_RL,
        #                              Direction_RR,Direction_RS,Direction_SL,Direction_SR, Direction_SS]])
        #
        # Y : array([Parkinsons,......])
        return X, Y

#x_path = "F:/parkisons/code/processed_dataset/furtherProcessed_tappy_data/"
#y_path = "F:/parkisons/code/processed_dataset/User_details.xlsx"
#X, Y = readXY(x_path,y_path)

#Y_dict = {}
#for row in Y:
    #print(row[0],row)
#    Y_dict[row[0]]= row

#for i in Y_dict.keys():
#    continue
    #print(i,":",Y_dict[i])

#x_inp =X[72:73]
#print(x_inp)        
#X,Y = TextProcess(x_inp,Y_dict,x_path)
#print(X)
#print("\n\n\n",Y)
#print(len(X))
#print(len(Y))
#print("dtype: ",X[0].dtype)
#print(type(X[0]))




#gen = CustomBatchGenerator(X,Y,4)